In [ ]:
try:
  import datasets, evaluate, accelerate
  import gradio as gr
except ModuleNotFoundError:
  !pip install -U datasets evaluate accelerate gradio
  import datasets, evaluate, accelerate
  import gradio as gr

import random

import numpy as np
import pandas as pd

import torch
import transformers

print(f"Using transformers version: {transformers.__version__}")
print(f"Using datasets version: {datasets.__version__}")
print(f"Using torch version: {torch.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2

In [ ]:
sentences = {
    "अवधि": ["तपाईंको परियोजनाको अवधि तीन महिना छ।"],
    "अवधी": ["अवधी लामो भएको कारण परियोजना ढिला भयो।"],
    "आँत": ["मैले त्यस्तो भयरहित आँत देखेको थिएन।"],
    "आँसु": ["उसका आँसु उसका दुःखको गहिराइलाई देखाउँछन्।"],
    "आशु": ["आशु दिने कसम पनि पछि टूट्न सक्छ।"],
    "आदि": ["हामीले सबैकुरा समेट्ने कसम खाएका थियौं, आदि।"],
    "आवास": ["उसको घरको आवास आरामदायक थियो।"],
    "आभास": ["हाम्रो बीचको भिन्नता आभास भइरहेको थियो।"],
    "आशा": ["मैले भविष्यमा राम्रो आशा राखेकी छु।"],
    "आसा": ["उसको सफलता र आशा सँगसँगै बढ्दै गयो।"],
    "ओट": ["हामीले ओट लिएको छ र भोकमरीबाट बच्न सफल भएका छौं।"],
    "कीर्ति": ["उसको कीर्ति सबैलाई प्रोत्साहन दिने छ।"],
    "कृति": ["उसको कृति साहित्यिक जगतमा धेरै चर्चित छ।"],
    "कूल": ["कूल कामको मापदण्ड त्यहाँको संस्कृति छ।"],
    "कुल": ["उसको कुल परिवारमा सबैजना शुभचिन्तक छन्।"],
    "कोट": ["मैले सस्तो कोट किनें।"],
    "कोत": ["मैले टमाटरको कोत बनायो।"],
    "कोश": ["उसको कोश भनेको शिक्षा क्षेत्रको विकास हो।"],
    "कोस": ["उपलब्धिहरूको कोस देखिनु भएको छ।"],
    "क्षत्रिय": ["क्षत्रिय जाति प्राचीन भारतीय समाजमा उच्च स्थानमा थियो।"],
    "क्षेत्रीय": ["क्षेत्रीय समस्याहरूलाई सुल्झाउन स्थानिय स्तरमा ध्यान दिनु पर्छ।"],
    "खाट": ["उनको खाट बलियो थियो।"],
    "खालि": ["तपाईंको जागिरको ठाउँ खालि छ।"],
    "खाली": ["सपना खाली छैन, तिमीलाई यसलाई पछ्याउनु पर्छ।"],
    "गधा": ["गधा एक परम्परागत काम गर्ने जनावर हो।"],
    "गदा": ["उसको गदा प्राचीन इतिहासमा महत्त्वपूर्ण वस्तु हो।"],
    "गर्भ": ["तपाईंको गर्भावस्था शुभ र स्वस्थ रहोस्।"],
    "घरी": ["हामी घरी घरी एकअर्कालाई सम्झन्छौं।"],
    "घडी": ["उसको घडी हराएको छ।"],
    "चिन": ["उसको चिन चिन्ता र दुःखबाट थम्याउने छ।"],
    "चिर": ["उसको चिर अनलाइन नेटवर्कमा फैलिएको छ।"],
    "चीर": ["उसको चीर देखेर सबै स्तब्ध भए।"],
    "जाति": ["जाति र धर्मको भेदभाव हामीले हटाउनै पर्छ।"],
    "जुन": ["जुन महिनामा म यहाँ आउँछु।"],
    "जून": ["जून महिनामा तापक्रम धेरै बढ्छ।"],
    "ताप": ["शरद ऋतुमा ताप फरक हुन सक्छ।"],
    "टार": ["मैले टार लागिरहेको ठाउँमा ट्रक रोके।"],
    "दर": ["बैंकको दर बढेको छ।"],
    "तातो": ["तातो पानी पिउँदा शरीरलाई आराम मिल्छ।"],
    "टाटो": ["उसको टाटो ड्युटी भरिरहेको थियो।"],
    "तिन": ["तिन जनाले नयाँ परियोजना सँगै काम गरे।"],
    "तीन": ["तीन पंक्ति छन्, जसको बीचमा एउटा मद्दत छ।"],
    "तिर": ["पढाइमा ध्यान तिर राख्नुहोस्।"],
    "तीर": ["तीर कसमलाई प्रयोग गर्दै उनी सिधा बढेका थिए।"],
    "तेस्रो": ["तेस्रो स्थानको जितले मलाई खुसी बनायो।"],
    "तेर्सो": ["उसको सोचमा तेर्सो मोड राख्न चाहन्छु।"],
    "थाम": ["तपाईंको थाम सँगै स्थितिहरूलाई नियन्त्रण गर्नु पर्छ।"],
    "ठाम": ["उसको भट्ठीमा एक ठाम राख्न पर्छ।"],
    "दण्ड": ["उसलाई दण्ड दिइएको छ।"],
    "दिन": ["मलाई हरेक दिन अभ्यास गर्न मन छ।"],
    "दीन": ["उसको दीन देखेर म दुखी भएँ।"],
    "दिल": ["उसको दिल ठूलो र माया गर्ने छ।"],
    "डिल": ["उसको डिल क्याच खेलका लागि अनिवार्य छ।"],
    "दिशा": ["तपाईंको दिशा स्पष्ट हो भने सफलता पाउन सजिलो हुन्छ।"],
    "दिसा": ["त्यो किस्सा सबको दिसा भनेको हुँदैन।"],
    "दीप": ["दीप जलेर अन्धकारको अन्त्य हुन्छ।"],
    "देखी": ["तपाईंको कामको स्थितिको निगरानी गर्न मैले देखी राखेको छु।"],
    "देखि": ["सपना देखी सुरु भएर यथार्थ सम्म पुग्न प्रयास गरौं।"],
    "द्वेष": ["हामी द्वेष बिना समाजमा सहकार्य गर्न सक्छौं।"],
    "द्वैत": ["द्वैत भक्ति वा वेदांत दर्शनका आधारमा व्याख्या गरिएको छ।"],
    "द्वैध": ["द्वैध दृषटिको विशेषता भनेको असमझदारी हो।"],
    "धाक": ["उसको धाक अडिग र अपरिहार्य छ।"],
    "नसा": ["तपाईंको नसा स्वास्थ्यको संकेत हो।"],
    "नशा": ["अत्यधिक नशा नकरात्मक प्रभाव पार्न सक्छ।"],
    "नाउ": ["उनको नाउ पहिले मेरो साथीको थियो।"],
    "नाउँ": ["तपाईंको नाउँ राम्रो र चर्चित छ।"],
    "निर्वाण": ["निर्वाण प्राप्त गर्नु बुद्ध धर्मको मुख्य उद्देश्य हो।"],
    "निर्माण": ["परियोजनाको निर्माण प्रक्रियामा सन्देश प्रेषित गर्ने छ।"],
    "पद": ["तपाईंको पद सँगै कर्तव्य पनि बढ्छ।"]
}

In [ ]:
"द्वैत भक्ति वा वेदांत दर्शनका आधारमा व्याख्या गरिएको छ।" == "द्वैध भक्ति वा वेदांत दर्शनका आधारमा व्याख्या गरिएको छ।"

False

In [ ]:
# confusion_set.json (Example)
confusion_set = {
    "नाम": ["नाउँ"],
    "नाउँ": ["नाम"],
    "फुल": ["फूल"],
    "फूल": ["फूल"],
    "नाडी": ['नदी'],
    'नदी': ['नाडी'],
    "अवधि": ["अवधी"],
    "अवधी": ["अवधि"],
    "आँट": ["आँत"],
    "आँत": ["आँट"],
    "आँसु": ["आशु"],
    "आशु": ["आँसु"],
    "आदि": ["आँधी"],
    "आँधी": ["आदि"],
    "आवास": ["आभास"],
    "आभास": ["आवास"],
    "आशा": ["आसा"],
    "आसा": ["आशा"],
    "ओट": ["ओठ"],
    "ओठ": ["ओट"],
    "काठ": ["काट"],
    "काट": ["काठ"],
    "कीर्ति": ["कृति"],
    "कृति": ["कीर्ति"],
    "कूल": ["कुल"],
    "कुल": ["कूल"],
    "कोट": ["कोत"],
    "कोत": ["कोट"],
    "कोश": ["कोस"],
    "कोस": ["कोश"],
    "क्षत्रिय": ["क्षेत्रीय"],
    "क्षेत्रीय": ["क्षत्रिय"],
    "खाग": ["खाक"],
    "खाक": ["खाग"],
    "खात": ["खाट"],
    "खाट": ["खात"],
    "खालि": ["खाली"],
    "खाली": ["खालि"],
    "गधा": ["गदा"],
    "गदा": ["गधा"],
    "गर्व": ["गर्भ"],
    "गर्भ": ["गर्व"],
    "घट": ["घत"],
    "घत": ["घट"],
    "घरी": ["घडी"],
    "घडी": ["घरी"],
    "घाट": ["घात"],
    "घात": ["घाट"],
    "चिन": ["चीन"],
    "चीन": ["चिन"],
    "चिर": ["चीर"],
    "चीर": ["चिर"],
    "जाति": ["जाती"],
    "जाती": ["जाति"],
    "जुन": ["जून"],
    "जून": ["जुन"],
    "टाप": ["ताप"],
    "ताप": ["टाप"],
    "टार": ["तार"],
    "तार": ["टार"],
    "डर": ["दर"],
    "दर": ["डर"],
    "डाम": ["दाम"],
    "दाम": ["डाम"],
    "तातो": ["टाटो"],
    "टाटो": ["तातो"],
    "तिन": ["तीन"],
    "तीन": ["तिन"],
    "तिर": ["तीर"],
    "तीर": ["तिर"],
    "तेस्रो": ["तेर्सो"],
    "तेर्सो": ["तेस्रो"],
    "थाम": ["ठाम"],
    "ठाम": ["थाम"],
    "दण्ड": ["द्वन्द्व"],
    "द्वन्द्व": ["दण्ड"],
    "दिन": ["दीन"],
    "दीन": ["दिन"],
    "दिल": ["डिल"],
    "डिल": ["दिल"],
    "दिशा": ["दिसा"],
    "दिसा": ["दिशा"],
    "दीप": ["द्विप"],
    "द्विप": ["दीप"],
    "देखी": ["देखि"],
    "देखि": ["देखी"],
    "देश": ["द्वेष"],
    "द्वेष": ["देश"],
    "द्वैत": ["द्वैध"],
    "द्वैध": ["द्वैत"],
    "धाक": ["ढाक"],
    "ढाक": ["धाक"],
    "नसा": ["नशा"],
    "नशा": ["नसा"],
    "नाउ": ["नाउँ"],
    "नाउँ": ["नाउ"],
    "निर्वाण": ["निर्माण"],
    "निर्माण": ["निर्वाण"],
    "पट्टि": ["पट्टी"],
    "पट्टी": ["पट्टि"],
    "पद": ["पथ"],
    "पथ": ["पद"],
    "परिणाम": ["परिमाण"],
    "परिमाण": ["परिणाम"],
    "पाइन्": ["पाइन"],
    "पाइन": ["पाइन्"],
    "पाटो": ["पातो"],
    "पातो": ["पाटो"],
    "पात": ["पाठ"],
    "पाठ": ["पात"],
    "पालि": ["पाली"],
    "पाली": ["पालि"],
    "पीठ": ["पिट"],
    "पिट": ["पीठ"],
    "पीडा": ["पिडा"],
    "पिडा": ["पीडा"],
    "पूरा": ["पुरा"],
    "पुरा": ["पूरा"],
    "पौवा": ["पौबा"],
    "पौबा": ["पौवा"],
    "प्रदान": ["प्रधान"],
    "प्रधान": ["प्रदान"],
    "फुल": ["फूल"],
    "फूल": ["फुल"],
    "फेरि": ["फेरी"],
    "फेरी": ["फेरि"],
    "बन्चरो": ["बनचरो"],
    "बनचरो": ["बन्चरो"],
    "बन्द": ["बन्ध"],
    "बन्ध": ["बन्द"],
    "बन्दा": ["बन्डा"],
    "बन्डा": ["बन्दा"],
    "बाँस": ["वास"],
    "वास": ["बाँस"],
    "बाघ": ["बाग"],
    "बाग": ["बाघ"],
    "बाजी": ["बाजि"],
    "बाजि": ["बाजी"],
    "बाण": ["बान"],
    "बान": ["बाण"],
    "बाधा": ["वादा"],
    "वादा": ["बाधा"],
    "बार": ["बाह्र"],
    "बाह्र": ["बार"],
    "बिना": ["वीणा"],
    "वीणा": ["बिना"],
    "बेस": ["वेश"],
    "वेश": ["बेस"],
    "बोध": ["बोझ"],
    "बोझ": ["बोध"],
    "ब्रह्मा": ["बर्मा"],
    "बर्मा": ["ब्रह्मा"],
    "भए": ["भय"],
    "भय": ["भए"],
    "भाँडा": ["भाडा"],
    "भाडा": ["भाँडा"],
    "महिना": ["मैना"],
    "मैना": ["महिना"],
    "मुद्रा": ["मुर्दा"],
    "मुर्दा": ["मुद्रा"],
    "यस": ["यश"],
    "यश": ["यस"],
    "रङ": ["रङ्ग"],
    "रङ्ग": ["रङ"],
    "लता": ["लटा"],
    "लटा": ["लता"],
    "लेक": ["लेख"],
    "लेख": ["लेक"],
    "वर": ["बर"],
    "बर": ["वर"],
    "वश": ["बस"],
    "बस": ["वश"],
    "वस्तु": ["बस्तु"],
    "बस्तु": ["वस्तु"],
    "विचार": ["बिचार"],
    "बिचार": ["विचार"],
    "विष": ["बिस"],
    "बिस": ["विष"],
    "विहार": ["बिहार"],
    "बिहार": ["विहार"],
    "शठ": ["षट्"],
    "षट्": ["शठ"],
    "शर": ["सर"],
    "सर": ["शर"],
    "शाला": ["साला"],
    "साला": ["शाला"],
    "संवेदना": ["समवेदना"],
    "समवेदना": ["संवेदना"],
    "सबल": ["शबल"],
    "शबल": ["सबल"],
    "सब": ["शव"],
    "शव": ["सब"],
    "सव्य": ["सभ्य"],
    "सभ्य": ["सव्य"],
    "सित": ["शीत"],
    "शीत": ["सित"],
    "सिता": ["सीता"],
    "सीता": ["सिता"],
    "सुची": ["शुचि"],
    "शुचि": ["सुची"],
    "सुर": ["शूर"],
    "शूर": ["सुर"],
    "सूची": ["सूचि"],
    "सूचि": ["सूची"],
    "स्त्री": ["इस्त्री"],
    "इस्त्री": ["स्त्री"],
    "हात": ["हाट"],
    "हाट": ["हात"],
    "भोक": ["भोग"],
    "भोग": ["भोक"]
}



In [ ]:
test_set = {}
for key, value  in sentences.items():
  new_sent = value[0].replace(key, confusion_set[key][0])
  test_set[key] = [*value, new_sent]

In [ ]:
len(test_set)

69

In [ ]:
test_set

{'अवधि': ['तपाईंको परियोजनाको अवधि तीन महिना छ।',
  'तपाईंको परियोजनाको अवधी तीन महिना छ।'],
 'अवधी': ['अवधी लामो भएको कारण परियोजना ढिला भयो।',
  'अवधि लामो भएको कारण परियोजना ढिला भयो।'],
 'आँत': ['मैले त्यस्तो भयरहित आँत देखेको थिएन।',
  'मैले त्यस्तो भयरहित आँट देखेको थिएन।'],
 'आँसु': ['उसका आँसु उसका दुःखको गहिराइलाई देखाउँछन्।',
  'उसका आशु उसका दुःखको गहिराइलाई देखाउँछन्।'],
 'आशु': ['आशु दिने कसम पनि पछि टूट्न सक्छ।',
  'आँसु दिने कसम पनि पछि टूट्न सक्छ।'],
 'आदि': ['हामीले सबैकुरा समेट्ने कसम खाएका थियौं, आदि।',
  'हामीले सबैकुरा समेट्ने कसम खाएका थियौं, आँधी।'],
 'आवास': ['उसको घरको आवास आरामदायक थियो।', 'उसको घरको आभास आरामदायक थियो।'],
 'आभास': ['हाम्रो बीचको भिन्नता आभास भइरहेको थियो।',
  'हाम्रो बीचको भिन्नता आवास भइरहेको थियो।'],
 'आशा': ['मैले भविष्यमा राम्रो आशा राखेकी छु।',
  'मैले भविष्यमा राम्रो आसा राखेकी छु।'],
 'आसा': ['उसको सफलता र आशा सँगसँगै बढ्दै गयो।',
  'उसको सफलता र आशा सँगसँगै बढ्दै गयो।'],
 'ओट': ['हामीले ओट लिएको छ र भोकमरीबाट बच्न सफल भएका छौं।',
  'ह

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import json
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")
model = AutoModelForMaskedLM.from_pretrained("Shushant/nepaliBERT")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load Confusion Set
# with open("confusion_set.json", "r", encoding="utf-8") as f:
#     confusion_set = json.load(f)
;

config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/529k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


''

In [ ]:
confusion_set.get('फुल')

['फूल']

In [ ]:
from nepalitokenizers import WordPiece

# importing from the HuggingFace tokenizers package
from tokenizers.processors import TemplateProcessing

text = "द्वैध भक्ति वा वेदांत दर्शनका आधारमा व्याख्या गरिएको छ।"

tokenizer_sp = WordPiece()

# using default post processor
tokens = tokenizer_sp.encode(text)
print(tokens.tokens)

# change the post processor to not add any special tokens
# treat tokenizer_sp as HuggingFace's Tokenizer object
tokenizer_sp.post_processor = TemplateProcessing()

tokens = tokenizer_sp.encode(text)
print(tokens.tokens)
# 'तीन जनाले नयाँ परियोजना सँगै काम गरे

['[CLS]', 'द्व', '##ैध', 'भक्ति', 'वा', 'वेद', '##ा', '##ं', '##त', 'दर्शन', '##का', 'आधारमा', 'व्याख्या', 'गरिएको', 'छ', '।', '[SEP]']
['द्व', '##ैध', 'भक्ति', 'वा', 'वेद', '##ा', '##ं', '##त', 'दर्शन', '##का', 'आधारमा', 'व्याख्या', 'गरिएको', 'छ', '।']


In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer(text).input_ids)

['[CLS]',
 'द',
 '##व',
 '##ध',
 'भ',
 '##कत',
 '##ि',
 'वा',
 'व',
 '##दा',
 '##त',
 'दर',
 '##शनका',
 'आधारमा',
 'वय',
 '##ा',
 '##ख',
 '##या',
 'गरिएको',
 'छ',
 '।',
 '[SEP]']

In [ ]:
def tokenize_sentence(sentence):
    """Tokenizes and encodes the input sentence."""
    return tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)

def get_confusion_candidates(word):
    """Returns the confusion set for a word."""
    return confusion_set.get(word, [])

def compute_probabilities(sentence, word, candidates):
    """Computes probabilities for candidate words in the given sentence."""
    masked_sentence = sentence.replace(word, "[MASK]")
    inputs = tokenize_sentence(masked_sentence)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        mask_index = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

        softmax = torch.nn.functional.softmax(logits[0, mask_index, :], dim=-1)
        probs = {candidate: softmax[0, tokenizer.convert_tokens_to_ids(candidate)].item() for candidate in candidates}

    return probs


In [ ]:
def spell_check(sentence):
    """Checks and corrects spelling for a given Nepali sentence."""
    words = sentence.split()
    corrected_sentence = []

    for word in words:
        if word in confusion_set:
            candidates = get_confusion_candidates(word)
            if candidates:
                probs = compute_probabilities(sentence, word, [word, *candidates])
                print(probs)
                best_candidate = max(probs, key=probs.get)
                corrected_sentence.append(best_candidate)
            else:
                corrected_sentence.append(word)
        else:
            corrected_sentence.append(word)

    return " ".join(corrected_sentence)


In [ ]:
if __name__ == "__main__":
    input_sentence = "तपाईंको परियोजनाको अवधी तीन महिना छ।"
    corrected_sentence = spell_check(input_sentence)
    print("Input Sentence:", input_sentence)
    print("Corrected Sentence:", corrected_sentence)


{'अवधी': 0.00022857954900246114, 'अवधि': 0.3219529986381531}
{'तीन': 0.11985927075147629, 'तिन': 0.003173626260831952}
{'महिना': 0.37227457761764526, 'मैना': 1.5016489385288878e-08}
Input Sentence: तपाईंको परियोजनाको अवधी तीन महिना छ।
Corrected Sentence: तपाईंको परियोजनाको अवधि तीन महिना छ।


In [ ]:
i =0
for key, value in test_set.items():
  input_sentence = value[1]
  corrected_sentence = spell_check(input_sentence)
  print("Input Sentence:", input_sentence)
  print("Corrected Sentence:", corrected_sentence)
  if corrected_sentence == value[0]:
    print('✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓')
    i+=1
  else:
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

{'अवधी': 0.00022857954900246114, 'अवधि': 0.3219529986381531}
{'तीन': 0.11985927075147629, 'तिन': 0.003173626260831952}
{'महिना': 0.37227457761764526, 'मैना': 1.5016489385288878e-08}
Input Sentence: तपाईंको परियोजनाको अवधी तीन महिना छ।
Corrected Sentence: तपाईंको परियोजनाको अवधि तीन महिना छ।
✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
{'अवधि': 0.02321249060332775, 'अवधी': 7.062517397571355e-05}
Input Sentence: अवधि लामो भएको कारण परियोजना ढिला भयो।
Corrected Sentence: अवधि लामो भएको कारण परियोजना ढिला भयो।
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
{'आँट': 4.925859542481703e-08, 'आँत': 9.407844459019543e-08}
Input Sentence: मैले त्यस्तो भयरहित आँट देखेको थिएन।
Corrected Sentence: मैले त्यस्तो भयरहित आँत देखेको थिएन।
✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
{'आशु': 9.799738798221824e-09, 'आँसु': 7.541686386502988e-08}
Input Sentence: उसका आशु उसका दुःखको गहिराइलाई देखाउँछन्।
Corrected Sentence: उसका आँसु उसका दुःखको गहिराइलाई देखाउँछन्।
✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
{'आँसु': 8.971755960374139e-08, 'आशु': 1.9176221499606072e-08}
Input Sentence: 

In [ ]:
i/len(test_set)

0.5507246376811594

In [ ]:
pip install nepalitokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.2/678.2 kB 8.4 MB/s eta 0:00:00


['[CLS]', 'हाम्रो', 'मातृ', '##भूमि', 'नेपाल', 'हो', '[SEP]']
['हाम्रो', 'मातृ', '##भूमि', 'नेपाल', 'हो']
